In [ ]:
import pandas as pd

# Läser in fliken "Veckodata Riket" från datafilen "fhm_covid19" .
df_covid19 = pd.read_excel('Data/fhm_covid19.xlsx', sheet_name='Veckodata Riket')

# En initial dataanalys av "Veckodata Riket"
print(df_covid19.info())
print(df_covid19.describe())
print(df_covid19['Antal_fall_100000inv_14dagar'].value_counts())
print(df_covid19.head())
print(df_covid19.columns)
print(df_covid19.index)

In [ ]:
# Slår ihop kolumnerna "år" och "veckonummer" till en "Vecka"-kolumn.
df_covid19['Vecka'] = df_covid19['år'].astype(str) + "v" + df_covid19['veckonummer'].astype(str)

print(df_covid19.head())

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Förbereder subplottens storlek
fig, ax = plt.subplots(figsize=(7, 4))

# Anpassar antalet etiketter som visas på x-axeln för att undvika överlappning
ticks_to_show = df_covid19['Vecka'].iloc[::3]

# Skapar ett subplot-linjediagram över antalet avlidna fall per vecka
sns.lineplot(data=df_covid19, x='Vecka', y='Antal_avlidna_vecka', ax=ax)
ax.set_xticks(ticks_to_show.index)
ax.set_xticklabels(ticks_to_show.values, fontsize='x-small', rotation=90) # Justerar x-axel etiketter för läsbarhet
ax.set_title("Antal avlidna fall per vecka")
ax.set_xlabel('Vecka')
ax.set_ylabel('Antal avlidna')
plt.tight_layout() # Anpassar layout:en så att allt får plats i den sparade bilden
plt.savefig('Visualisering/avlidna_per_vecka.png', facecolor='w') # Sparar diagrammet & sätter bakgrundsfärgen till vit
plt.show()

In [ ]:
import plotly.express as px

# Skapar ett linjediagram
fig = px.line(df_covid19, x='Vecka', y='Antal_avlidna_vecka', title='Antal avlidna fall per vecka')

# Förbättrar layouten
fig.update_layout(
    xaxis_title='Vecka',
    yaxis_title='Antal avlidna',
    xaxis=dict(
        tickmode='array',
        tickvals=df_covid19['Vecka'][::3]), # Anpassar antalet etiketter som visas på x-axeln för att undvika överlappning 

    xaxis_tickangle= -90  # Roterar etiketterna på x-axeln för bättre läsbarhet
)

fig.show()

# Sparar grafen som en HTML-fil
fig.write_html('Visualisering/avlidna_per_vecka.html')